# 1. Installing Dependencies

In [ ]:
!git clone https://github.com/leoperezz/ObjectDetection

Cloning into 'ObjectDetection'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 56 (delta 15), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (56/56), done.


In [ ]:
!git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
!pip install tensorflow==2.7.0

# 2. Download training data
* We are going to download training data from Kaggle.

In [ ]:
!pip install opendatasets

In [ ]:
import opendatasets as od


cstrike='https://www.kaggle.com/datasets/lstmkirigaya/cstrike-detection'
traffic_detection='https://www.kaggle.com/datasets/saumyapatel/traffic-vehicles-object-detection'

od.download('https://www.kaggle.com/datasets/lstmkirigaya/cstrike-detection')

In [ ]:
from ObjectDetection.utils import reshape_img_and_bboxes
import numpy as np
from glob import glob
from os.path import join
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array


def process_label_array(label_array):
  '''
  This is for YOLOV5 format
  converts (label,x_center,y_center,w,h) to (label,ymin,xmin,ymax,xmax)
  '''
  real_shape=(int(len(label_array)/5),5)
  array=np.reshape(label_array,real_shape)
  labels_,bboxes_=array[:,0],array[:,1:]
  x_c,y_c,w,h=bboxes_[:,0],bboxes_[:,1],bboxes_[:,2],bboxes_[:,3]
  ymin,xmin,ymax,xmax=y_c-h/2,x_c-w/2,y_c+h/2,x_c+w/2
  final_array=np.stack([labels_,ymin,xmin,ymax,xmax])
  return final_array.T

def get_name_txt(path):
  '''
  Get the name from a file .txt in a path  
  '''
  x=path.split('/')
  x=x[-1].split('.')
  x=x[0]
  return x

def get_array_from_img(img_path):
  img=Image.open(img_path)
  img=img_to_array(img)
  if img.shape[-1]==4:
    img=Image.open(img_path)
    img=img.convert('RGB')
    img=img_to_array(img)  
  return img

def process_labels(labels,num_classes):
  labels_=tf.one_hot(labels,num_classes)
  labels_=tf.convert_to_tensor(labels_)
  return labels_


def handling_png_jpg(images_path,name_img):

    try:
      img_path=join(images_path,name_img)+'.jpg'
      img_array=get_array_from_img(img_path)
      return img_path,img_array
    except:
      img_path=join(images_path,name_img)+'.png'
      img_array=get_array_from_img(img_path)
      return img_path,img_array

def create_dataset(images_path,labels_path,target_size,batch_size,num_classes,size=0.9):
  
  assert size<=1, 'size must be less or equal than 1'

  label_glob=glob(labels_path+'/*.txt')

  size_cut=int(len(label_glob)*size)

  label_glob=label_glob[:size_cut]

  images_list,labels_list,bboxes_list=[],[],[]

  for label_path in label_glob:
    array=np.fromfile(label_path,sep=' ',dtype='float32')
    array=process_label_array(array)
    num_objects=array.shape[0]
    labels,bboxes=array[:,0],array[:,1:]
    labels=process_labels(labels,num_classes)
    
    name_img=get_name_txt(label_path)

    img_path,img_array=handling_png_jpg(images_path,name_img)

    img_array,bboxes=reshape_img_and_bboxes(img_array,bboxes,target_size)  
    for i in range(num_objects):
      images_list.append(img_array)
      labels_list.append(labels[i,:])
      bboxes_list.append(bboxes[i,:]) 

  img_tensor=np.stack(images_list)
  bboxes_tensor=np.stack(bboxes_list).astype('float32')
  labels_tensor=np.stack(labels_list).astype('float32')

  data=tf.data.Dataset.from_tensor_slices((img_tensor,bboxes_tensor,labels_tensor))
  data=data.batch(batch_size).shuffle(len(label_glob))
  data=[[img,bboxes,classes] for (img,bboxes,classes) in data]
  print(f'Data Created! size of the data {len(data)*batch_size}')
  return data

In [ ]:
BATCH_SIZE=32
target_size=(224,224)

'''For cstrike detection'''

images_path='/content/cstrike-detection/images'

labels_path='/content/cstrike-detection/labels'

num_classes=5

size=1
t_size=0.9
data=create_dataset(images_path,labels_path,target_size,5,num_classes,1)
size_train_data=int(len(data)*t_size)
train_data,val_data=data[:size_train_data],data[size_train_data:]


# 3. Creating a model

In [ ]:
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!tar -xf ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
!mv ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint models/research/object_detection/test_data/

In [ ]:
config_path='/content/models/research/object_detection/configs/tf2/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.config'
ckpt_path='/content/models/research/object_detection/test_data/checkpoint/ckpt-0'

configs=config_util.get_configs_from_pipeline_file(config_path)
model_config=configs['model']
model_config.ssd.num_classes=num_classes
model_config.ssd.freeze_batchnorm=True
model=model_builder.build(model_config=model_config,is_training=True)

In [ ]:
box_predictor_tmp = tf.compat.v2.train.Checkpoint(
    _base_tower_layers_for_heads=model._box_predictor._base_tower_layers_for_heads,
    _box_prediction_head=model._box_predictor._box_prediction_head,
    )

model_tmp = tf.compat.v2.train.Checkpoint(
          _feature_extractor=model._feature_extractor,
          _box_predictor=box_predictor_tmp)
ckpt = tf.compat.v2.train.Checkpoint(model=model_tmp)
ckpt.restore(ckpt_path).expect_partial()
images,shapes=model.preprocess(tf.ones((1,640,640,3)))
prediction_dict=model.predict(images,shapes)
_=model.postprocess(prediction_dict,shapes)

In [ ]:
'''Looking the variables'''

for i in model.trainable_variables:
  print(i.name)

In [ ]:
prefixes_to_train = [
  'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead',
  'WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead']

trainable_variables = model.trainable_variables
to_fine_tune = []

for var in trainable_variables:
  if any([var.name.startswith(prefix) for prefix in prefixes_to_train]):
    to_fine_tune.append(var)

# 4. Training the model

* The shape of the data needs to be: 
* image:(batch_size,H,W,3)
* bboxes: (batch_size,4)
* labels: (batch_size,num_classes)


In [ ]:
from ObjectDetection.utils import get_train_step_func,get_val_func
from ObjectDetection.utils import train_on_ds,val_on_ds

optimizer=tf.keras.optimizers.SGD(1e-3,momentum=0.9)
train_func=get_train_step_func(model,to_fine_tune,optimizer)
val_func=get_val_func(model)

In [ ]:
epochs=50
history={'train_loss':[],'test_loss':[]}
loss_reference=np.inf
with tf.device('/GPU:0'):
  for epoch in range(1,epochs+1):
    train_loss=train_on_ds(train_func,train_data)
    test_loss=val_on_ds(val_func,val_data)
    if test_loss<loss_reference:
      loss_reference=test_loss
      ckpt=tf.train.Checkpoint(model=model)
      ckpt.save('ckpt_counter_strike')
    history['train_loss'].append(train_loss)
    history['test_loss'].append(test_loss)
    print(f'epoch:{epoch} train_loss:{train_loss} test_loss:{test_loss}')


In [ ]:
import matplotlib.pyplot as plt

fig=plt.figure()
fig.set_size_inches((10,10))
plt.plot(np.array(history['train_loss']),label='train loss')
plt.plot(np.array(history['test_loss']),label='test loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.title('train and validation loss')
plt.legend()
plt.show()
%matplotlib inline

# 5.Testing the model for two instances:
* Using a model with the less train loss
* Using a model with the less test loss

In [ ]:
from ObjectDetection.utils import create_frame_video
import os

frames_video_path='/content/FramesVideo'
video_path='/content/video_real_cs.mp4'
name_frame='FRAME'
create_frame_video(video_path,name_frame,frames_video_path)

In [ ]:
from tensorflow.keras.utils import load_img
from ObjectDetection.utils import create_array_from_images_path

test_data=create_array_from_images_path(frames_video_path,name_frame,target_size,size_min=1)

In [ ]:
from ObjectDetection.utils import make_category_index

classes=['N', 'C', 'D', 'T', 'W']
category_index=make_category_index(classes)

category_index

### 5.1. Model with less train loss

In [ ]:
from ObjectDetection.utils import create_images_postprocess,create_video

prediction_v1='/content/Predictions/v1'
name_frame='POST_FRAME'
create_images_postprocess(model,test_data,prediction_v1,name_frame,(10,20),0.3,category_index)
images_post_v1=create_array_from_images_path(prediction_v1,name_frame,(224,224),1)
create_video(images_post_v1,'video_cs_v1')

### 5.1. Model with less test loss

In [ ]:

model_ckpt=tf.train.Checkpoint(model=model)
model_ckpt.restore('/content/ckpt_counter_strike-1')
images,shapes=model.preprocess(tf.ones((1,640,640,3)))
prediction_dict=model.predict(images,shapes)
_=model.postprocess(prediction_dict,shapes)

prediction_v1='/content/Predictions/v2'
name_frame='POST_FRAME'
create_images_postprocess(model,test_data,prediction_v1,name_frame,(10,20),0.3,category_index)
images_post_v1=create_array_from_images_path(prediction_v1,name_frame,(224,224),1)
create_video(images_post_v1,'video_cs_v2')

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/video_cs_v1.mp4 /content/drive/MyDrive

In [ ]:
!cp /content/video_cs_v2.mp4 /content/drive/MyDrive